# Pre-Processing Pipeline Stage

In [4]:
from pyspark.ml.classification import LogisticRegression
from pyspark.sql               import SparkSession
from pyspark.ml.linalg         import Vectors
from pyspark.ml.regression     import LabeledPoint

ImportError: cannot import name 'LabeledPoint' from 'pyspark.ml.regression' (/opt/anaconda/lib/python3.7/site-packages/pyspark-2.4.4-py3.7.egg/pyspark/ml/regression.py)

In [ ]:
def parse(line):
    return LabledPoint()